In [ ]:
import matplotlib.pyplot as plt

image_name = "sample_data/IMG/left_2016_12_01_13_30_48_287.jpg"
img=plt.imread("sample_data/IMG/left_2016_12_01_13_30_48_287.jpg")
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
import csv
import numpy as np

driving_log_filename = 'sample_data/driving_log.csv'

center_images = []
steerings = []
with open(driving_log_filename) as driving_log:
    driving_log_reader = csv.DictReader(driving_log)
    for row in driving_log_reader:
        center_images.append(plt.imread('sample_data/' + row['center']))
        steerings.append([float(row['steering'])])
                            
X_train = np.array(center_images)
y_train = np.array(steerings)
    
print(X_train.shape)
print(y_train.shape)


In [ ]:
# adding left and right cameras

import matplotlib.pyplot as plt
import csv
import numpy as np

driving_log_filename = 'sample_data/driving_log.csv'

images = []
steerings = []
theta = 0.35
with open(driving_log_filename) as driving_log:
    driving_log_reader = csv.DictReader(driving_log)
    for row in driving_log_reader:
        
        steering = float(row['steering'])
        
        images.append(plt.imread('sample_data/' + row['center'].strip()))
        steerings.append([steering])
        
        images.append(plt.imread('sample_data/' + row['left'].strip()))
        steerings.append([steering + theta])
        
        images.append(plt.imread('sample_data/' + row['right'].strip()))
        steerings.append([steering - theta])
        
                            
X_train = np.array(images)
y_train = np.array(steerings)
    
print(X_train.shape)
print(y_train.shape)


In [1]:
import matplotlib.pyplot as plt
import csv
import numpy as np

driving_log_filename = 'sample_data/driving_log.csv'

driving_log = []

with open(driving_log_filename) as driving_log_file:
    driving_log_reader = csv.DictReader(driving_log_file)
    for line in driving_log_reader:
        driving_log.append(line)
        
        
row = driving_log[0]
print (row['left'])

 IMG/left_2016_12_01_13_30_48_287.jpg


In [35]:
import csv
from math import ceil, floor
from random import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import Sequence


# ref: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DrivingLogSequence(Sequence):
    
    def __init__ (self, driving_log, batch_size=32):
        
        self.driving_log = driving_log
        self.batch_size = batch_size
        
        self.theta = 0.35
        
        return
    
    def __len__(self):
        return floor(len(self.driving_log)/self.batch_size)
    
    def __getitem__(self, index):
        
        batch_driving_log = self.driving_log[index:index+self.batch_size]
        images = []
        steerings = []
        theta = 0.35
        
        for line in batch_driving_log:
            
            steering = float(line['steering'])

            images.append(plt.imread('sample_data/' + line['center'].strip()))
            steerings.append([steering])

            images.append(plt.imread('sample_data/' + line['left'].strip()))
            steerings.append([steering + theta])

            images.append(plt.imread('sample_data/' + line['right'].strip()))
            steerings.append([steering - theta])
            
        X = np.array(images)
        y = np.array(steerings)
        
        return X, y
    
    def on_epoch_end(self):
        shuffle(self.driving_log)
        return
        



In [36]:
driving_log_training, driving_log_validation = train_test_split(driving_log, test_size=0.2)
driving_log_seq_training   = DrivingLogSequence(driving_log_training)
driving_log_seq_validation = DrivingLogSequence(driving_log_validation)

In [ ]:
 
'''
def generator(lines, batch_size=32):

    driving_log_filename = 'sample_data/driving_log.csv'

    images = []
    steerings = []
    theta = 0.35
    
    with open(driving_log_filename) as driving_log:
        num_lines = 0
        for line in driving_log:
            num_lines += 1

    with open(driving_log_filename) as driving_log:

        driving_log_reader = csv.DictReader(driving_log)
        
        
        print (len(driving_log_reader))

    for row in driving_log_reader:

        steering = float(row['steering'])

        images.append(plt.imread('sample_data/' + row['center'].strip()))
        steerings.append([steering])

        images.append(plt.imread('sample_data/' + row['left'].strip()))
        steerings.append([steering + theta])

        images.append(plt.imread('sample_data/' + row['right'].strip()))
        steerings.append([steering - theta])
    '''
        

In [ ]:
plt.imshow(X_train[45])

In [ ]:
y_train[46]

for y in y_train:
    if y > 0:
        print(y)
        break

## Network Structure

In [31]:
from keras.models import Sequential
from keras.layers import Lambda, Flatten, Dense, Conv2D, Cropping2D


def normalize(rgb):
    '''
    normalize rgb between [-1, 1]
    '''
    
    return (rgb-128.0) / 128.0


model = Sequential()

model.add(Lambda(normalize, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=[(50, 20), (0, 0)]))
model.add(Conv2D(filters=24, kernel_size=5, strides=2))
model.add(Conv2D(filters=36, kernel_size=5, strides=2))
model.add(Conv2D(filters=48, kernel_size=5, strides=2))
model.add(Conv2D(filters=64, kernel_size=3, strides=2))
model.add(Conv2D(filters=64, kernel_size=3, strides=2))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
          

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 18, 64)         27712     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 1, 8, 64)          36928     
__________

In [33]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam()
model.compile(loss=tf.keras.losses.MSE, optimizer='Adam')

In [37]:
model.fit_generator(driving_log_seq_training, 
                    validation_data=driving_log_seq_validation, 
                    epochs=3)

Epoch 1/3
200/200 [==============================] - 21s 107ms/step - loss: 0.0071 - val_loss: 0.0558
Epoch 2/3
200/200 [==============================] - 20s 102ms/step - loss: 0.0058 - val_loss: 0.0577
Epoch 3/3
200/200 [==============================] - 21s 106ms/step - loss: 0.0069 - val_loss: 0.0510


In [ ]:
model.fit(X_train, y_train, batch_size=512, initial_epoch=0, epochs=4, shuffle=True, validation_split=0.2)

In [ ]:
model.save('model.h5')

# drive.py

In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )
    parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )
    #parser.parse_args()    
    #parser.parse_args(['--sum', '7', '-1', '42'])    
    args = parser.parse_args(['model.h5'])

    # check that model Keras version is same as local Keras version
    f = h5py.File(args.model, mode='r')
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    #mine
    print(args.model)
    model = load_model(args.model)

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


In [ ]:
import sys
print(sys.version)


In [ ]:
import socketio